In [1]:
%load_ext autoreload
%autoreload 2

from statsmodels.base.model import GenericLikelihoodModel

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats
import sys

from scipy.stats import multivariate_normal

sys.path.append("../")
import vuong_tests5

In [2]:
class JointNormal1(GenericLikelihoodModel):
    
    def loglikeobs(self, params):
        data = np.concatenate([[self.endog],self.exog.transpose()],axis=0)
        mult_rv = stats.multivariate_normal([params[0], 0.0], [[1,0],[0,1]])
        return mult_rv.logpdf(data.transpose())
    
    
class JointNormal2(GenericLikelihoodModel):
    
    def loglikeobs(self, params):
        data = np.concatenate([[self.endog],self.exog.transpose()],axis=0)
        mult_rv = stats.multivariate_normal([0.0, params[0]], [[1,0],[0,1]])
        return mult_rv.logpdf(data.transpose())


def setup_shi(yn,xn):
    # model 1 grad, etc.
    nobs = yn.shape[0]
    model1_param = np.array([yn.mean()])
    model2_param = np.array([xn.mean()])
    
    model1_deriv = JointNormal1(yn,xn)
    ll1 = model1_deriv.loglikeobs(model1_param)
    grad1 =  model1_deriv.score_obs(model1_param).reshape( (nobs,1) )
    hess1 = model1_deriv.hessian(model1_param)
    
    
    model2_deriv = JointNormal2(yn,xn)
    ll2 = model2_deriv.loglikeobs(model2_param)
    grad2 =  model2_deriv.score_obs(model2_param).reshape( (nobs,1) )  
    hess2 = model2_deriv.hessian(model2_param)
    
    return ll1,grad1,hess1,model1_param,ll2,grad2,hess2,model2_param

def gen_data(beta= 1.5, nobs=1000):
    cov = [[25, 0], [0, 1]]
    data = np.random.multivariate_normal([beta,beta], [[25,0],[0,1]],  nobs)
    return data[:,0],data[:,1],nobs

yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn)
print(grad1.shape,hess1.shape)

(1000, 1) (1, 1)


# Ex 0

In [3]:
beta = 0

In [4]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, beta =beta)
mc_out = vuong_tests5.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 1.00 & 1.00 & 0.76 & 0.74 & 0.75 & 0.89   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.00 & 0.00 & 0.24 & 0.26 & 0.25 & 0.11   \\
\hline
\end{tabular}
(array([1., 0., 0.]), array([1., 0., 0.]), array([0.76, 0.  , 0.24]), array([0.74, 0.  , 0.26]), array([0.75, 0.  , 0.25]), array([0.89, 0.  , 0.11]), 10.11464704417287, 14.064918491512161, 18.407127047757044)


In [5]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, beta =beta)
mc_out = vuong_tests5.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 1.00 & 1.00 & 0.79 & 0.80 & 0.80 & 0.91   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.00 & 0.00 & 0.21 & 0.20 & 0.20 & 0.09   \\
\hline
\end{tabular}
(array([1., 0., 0.]), array([1., 0., 0.]), array([0.79, 0.  , 0.21]), array([0.8, 0. , 0.2]), array([0.8, 0. , 0.2]), array([0.91, 0.  , 0.09]), 11.21360786264128, 16.06080302492774, 19.52022803811439)


In [6]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=1000, beta =beta)
mc_out = vuong_tests5.monte_carlo(100,gen_data_ex,setup_shi)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 1.00 & 1.00 & 0.83 & 0.83 & 0.83 & 0.87   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.00 & 0.00 & 0.17 & 0.17 & 0.17 & 0.13   \\
\hline
\end{tabular}
(array([1., 0., 0.]), array([1., 0., 0.]), array([0.83, 0.  , 0.17]), array([0.83, 0.  , 0.17]), array([0.83, 0.  , 0.17]), array([0.87, 0.  , 0.13]), 11.788164813176138, 14.95395236654706, 20.213103713017023)


# Ex 0.5

In [19]:
beta = 0.5

In [20]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, beta =beta)
mc_out = vuong_tests5.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.90 & 0.90 & 0.81 & 0.81 & 0.79 & 0.93   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.10 & 0.10 & 0.19 & 0.19 & 0.21 & 0.07   \\
\hline
\end{tabular}
(array([0.9, 0. , 0.1]), array([0.9, 0. , 0.1]), array([0.81, 0.  , 0.19]), array([0.81, 0.  , 0.19]), array([0.79, 0.  , 0.21]), array([0.93, 0.  , 0.07]), 4.504496668798725, 33.06173612975241, 38.88676392787596)


In [21]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, beta =beta)
mc_out = vuong_tests5.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.93 & 0.93 & 0.91 & 0.87 & 0.91 & 0.96   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.01 & 0.00 & 0.00   \\
Model 2 & 0.07 & 0.07 & 0.09 & 0.12 & 0.09 & 0.04   \\
\hline
\end{tabular}
(array([0.93, 0.  , 0.07]), array([0.93, 0.  , 0.07]), array([0.91, 0.  , 0.09]), array([0.87, 0.01, 0.12]), array([0.91, 0.  , 0.09]), array([0.96, 0.  , 0.04]), 8.628479823933413, 56.195149379589665, 56.94517527583182)


In [22]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=1000, beta =beta)
mc_out = vuong_tests5.monte_carlo(100,gen_data_ex,setup_shi)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.93 & 0.93 & 0.92 & 0.92 & 0.92 & 0.98   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.07 & 0.07 & 0.08 & 0.08 & 0.08 & 0.02   \\
\hline
\end{tabular}
(array([0.93, 0.  , 0.07]), array([0.93, 0.  , 0.07]), array([0.92, 0.  , 0.08]), array([0.92, 0.  , 0.08]), array([0.92, 0.  , 0.08]), array([0.98, 0.  , 0.02]), 11.595935364365623, 81.8114927479622, 81.16773243419924)


# Ex 1

In [7]:
beta = 1

In [8]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, beta =beta)
mc_out = vuong_tests5.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.93 & 0.93 & 0.91 & 0.93 & 0.92 & 0.98   \\
Model 1 & 0.01 & 0.01 & 0.02 & 0.01 & 0.02 & 0.00   \\
Model 2 & 0.06 & 0.06 & 0.07 & 0.06 & 0.06 & 0.02   \\
\hline
\end{tabular}
(array([0.93, 0.01, 0.06]), array([0.93, 0.01, 0.06]), array([0.91, 0.02, 0.07]), array([0.93, 0.01, 0.06]), array([0.92, 0.02, 0.06]), array([0.98, 0.  , 0.02]), 21.941139847932103, 86.95926263949372, 83.87079842985602)


In [9]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, beta =beta)
mc_out = vuong_tests5.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.89 & 0.89 & 0.89 & 0.88 & 0.87 & 1.00   \\
Model 1 & 0.03 & 0.03 & 0.01 & 0.03 & 0.04 & 0.00   \\
Model 2 & 0.08 & 0.08 & 0.10 & 0.09 & 0.09 & 0.00   \\
\hline
\end{tabular}
(array([0.89, 0.03, 0.08]), array([0.89, 0.03, 0.08]), array([0.89, 0.01, 0.1 ]), array([0.88, 0.03, 0.09]), array([0.87, 0.04, 0.09]), array([1., 0., 0.]), 19.455662509545768, 123.66668043149896, 115.32464334294588)


In [10]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=1000, beta =beta)
mc_out = vuong_tests5.monte_carlo(100,gen_data_ex,setup_shi)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.92 & 0.92 & 0.92 & 0.90 & 0.91 & 1.00   \\
Model 1 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.00   \\
Model 2 & 0.07 & 0.07 & 0.07 & 0.09 & 0.08 & 0.00   \\
\hline
\end{tabular}
(array([0.92, 0.01, 0.07]), array([0.92, 0.01, 0.07]), array([0.92, 0.01, 0.07]), array([0.9 , 0.01, 0.09]), array([0.91, 0.01, 0.08]), array([1., 0., 0.]), -3.562906532026608, 152.89951484879006, 157.49184481125533)


# Ex 2

In [11]:
beta = 1.5

In [12]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, beta =beta)
mc_out = vuong_tests5.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.94 & 0.94 & 0.92 & 0.94 & 0.91 & 0.99   \\
Model 1 & 0.01 & 0.01 & 0.01 & 0.01 & 0.02 & 0.00   \\
Model 2 & 0.05 & 0.05 & 0.07 & 0.05 & 0.07 & 0.01   \\
\hline
\end{tabular}
(array([0.94, 0.01, 0.05]), array([0.94, 0.01, 0.05]), array([0.92, 0.01, 0.07]), array([0.94, 0.01, 0.05]), array([0.91, 0.02, 0.07]), array([0.99, 0.  , 0.01]), 23.465049826091835, 126.55145835968626, 122.98721555989428)


In [13]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, beta =beta)
mc_out = vuong_tests5.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.96 & 0.96 & 0.95 & 0.95 & 0.94 & 1.00   \\
Model 1 & 0.01 & 0.01 & 0.01 & 0.01 & 0.01 & 0.00   \\
Model 2 & 0.03 & 0.03 & 0.04 & 0.04 & 0.05 & 0.00   \\
\hline
\end{tabular}
(array([0.96, 0.01, 0.03]), array([0.96, 0.01, 0.03]), array([0.95, 0.01, 0.04]), array([0.95, 0.01, 0.04]), array([0.94, 0.01, 0.05]), array([1., 0., 0.]), 12.59004451048569, 155.2101957083813, 170.4399214349766)


In [14]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=1000, beta =beta)
mc_out = vuong_tests5.monte_carlo(100,gen_data_ex,setup_shi)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.94 & 0.94 & 0.92 & 0.94 & 0.93 & 0.99   \\
Model 1 & 0.01 & 0.01 & 0.02 & 0.01 & 0.02 & 0.00   \\
Model 2 & 0.05 & 0.05 & 0.06 & 0.05 & 0.05 & 0.01   \\
\hline
\end{tabular}
(array([0.94, 0.01, 0.05]), array([0.94, 0.01, 0.05]), array([0.92, 0.02, 0.06]), array([0.94, 0.01, 0.05]), array([0.93, 0.02, 0.05]), array([0.99, 0.  , 0.01]), 15.290301455035895, 252.41720768708146, 243.11392708702277)


# Ex 3

In [15]:
beta = 2

In [16]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, beta =beta)
mc_out = vuong_tests5.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.96 & 0.96 & 0.94 & 0.95 & 0.93 & 1.00   \\
Model 1 & 0.01 & 0.01 & 0.02 & 0.02 & 0.02 & 0.00   \\
Model 2 & 0.03 & 0.03 & 0.04 & 0.03 & 0.05 & 0.00   \\
\hline
\end{tabular}
(array([0.96, 0.01, 0.03]), array([0.96, 0.01, 0.03]), array([0.94, 0.02, 0.04]), array([0.95, 0.02, 0.03]), array([0.93, 0.02, 0.05]), array([1., 0., 0.]), 28.02120843799466, 166.3624467455012, 163.1958983023873)


In [17]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, beta =beta)
mc_out = vuong_tests5.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.92 & 0.92 & 0.91 & 0.91 & 0.92 & 0.99   \\
Model 1 & 0.03 & 0.03 & 0.04 & 0.03 & 0.03 & 0.00   \\
Model 2 & 0.05 & 0.05 & 0.05 & 0.06 & 0.05 & 0.01   \\
\hline
\end{tabular}
(array([0.92, 0.03, 0.05]), array([0.92, 0.03, 0.05]), array([0.91, 0.04, 0.05]), array([0.91, 0.03, 0.06]), array([0.92, 0.03, 0.05]), array([0.99, 0.  , 0.01]), 25.72488643586545, 240.44292606949625, 229.75676264486574)


In [18]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=1000, beta =beta)
mc_out = vuong_tests5.monte_carlo(100,gen_data_ex,setup_shi)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.94 & 0.94 & 0.93 & 0.93 & 0.93 & 1.00   \\
Model 1 & 0.02 & 0.02 & 0.02 & 0.02 & 0.02 & 0.00   \\
Model 2 & 0.04 & 0.04 & 0.05 & 0.05 & 0.05 & 0.00   \\
\hline
\end{tabular}
(array([0.94, 0.02, 0.04]), array([0.94, 0.02, 0.04]), array([0.93, 0.02, 0.05]), array([0.93, 0.02, 0.05]), array([0.93, 0.02, 0.05]), array([1., 0., 0.]), -12.864352708758515, 318.04963778450747, 322.58939257064463)
